In [1]:
import pandas as pd
import csv
from Plan import CreateSamples_renewed
import importlib

In [22]:
importlib.reload(CreateSamples_renewed)

<module 'Plan.CreateSamples_renewed' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples_renewed.py'>

In [23]:
path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\New_Testing_Protocol_Generalized.csv"
chem_path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Chemical Database.csv"

In [24]:
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [32]:
df = CreateSamples_renewed.concentration_from_linspace(plan, unity_filter=True)

In [37]:
component_units = plan['Component Concentration Units']
total_sample_amount_unit = plan['Sample Unit']
total_sample_amount = plan['Sample Amount']
component_units

['wtf', 'wtf', 'wtf', 'wtf', 'wtf']

In [40]:
for col, unit in zip(component_units,df):
    if total_sample_amount_unit == 'g':
        if unit == 'wtf':
            mass = total_sample_amount*df[col] # you could raise a notification on what info is needed for stock with wtf
        if unit == 'mgpermL'
    if total_sample_amount_unit =='mL':
        if unit == 'mgpermL':
            # so when being labeled in the dataframe just make it clear that it is a mass of the component
        if unit == 'volf'
            # make it clear it is a volume